# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [6]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "69o6i0JqwgpLM5a5v4NN"
client_secret = "dFeCQMyNQd"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
    #    response_body = response.read()
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())
    #    print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [7]:
#print(news_data)
print(len(news_data))
#print(news_data[0])
#print(news_data[0]['items'])
#print(len(news_data[0]['items']))
#print(news_data[0]['items'][2])
#print(news_data[0]['items'][7]['link'])

3


In [8]:
naver_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
        #print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)
        

# 사이트 확인하기에 편한 코드 구조.
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0010713700
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004967362
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002233331
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=031&aid=0000623480
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=421&aid=0005595080
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=008&aid=0004643409
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002529018


In [9]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [19]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('./driver/chromedriver.exe')

In [20]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

<ipython-input-20-326f55771db9>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(naver_news_link))):


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-20-326f55771db9>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(naver_news_link[n]))):


  0%|          | 0/1 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0010713700


  0%|          | 0/2 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004967362
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002233331


  0%|          | 0/4 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=031&aid=0000623480
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=421&aid=0005595080
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=008&aid=0004643409
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002529018
["요기요, 신입·경력 '자바 개발자' 공개 채용"]
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    기사내용 요약 17일까지 홈페이지 통해 접수 진행 기존 파이썬에서 자바까지 개발자 채용 범위 확대 [서울=뉴시스] 오동현 기자 = 딜리버리히어로 코리아는 국내 대표 배달앱 ‘요기요’가 신입 및 경력직 자바(Java) 개발자 공개 채용을 진행한다고 10일 밝혔다. 요기요는 뛰어난 역량을 갖춘 개발자를 확보하기 위해 코딩 언어를 확대해 기존 파이썬에서 자바까지 채용 범위를 확대키로 했다. 이를 통해 신규 사업 등을 자바 언어를 기반으로도 개발할 예정이다. 이번 채용은 신입 및 경력 자바 서버 개발자를 대상으로 진행되며, 자바 개발 경험과 요기요 서비스에 관심 있는 개발자라면 누구나 지원 가능하다. 모집 기간은 오는 17일까지로, 지원서 접수 후 온라인 코딩테스트가 진행된다. 이후 테스트 결과에 따라 면접 전형을 진행해 최종 합격자를 선발할 예정이다. 면접은 코로나 19를 고려해 비대면 방식으로 진행된다. 모집 과정 및 일정에 대한 자세한 내용은 딜리버리히어로 코리아 테크 홈페이지를 통해 확인 가능하다. 조현

In [21]:
print(naver_news_title[0])

["요기요, 신입·경력 '자바 개발자' 공개 채용"]


In [22]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    기사내용 요약 17일까지 홈페이지 통해 접수 진행 기존 파이썬에서 자바까지 개발자 채용 범위 확대 [서울=뉴시스] 오동현 기자 = 딜리버리히어로 코리아는 국내 대표 배달앱 ‘요기요’가 신입 및 경력직 자바(Java) 개발자 공개 채용을 진행한다고 10일 밝혔다. 요기요는 뛰어난 역량을 갖춘 개발자를 확보하기 위해 코딩 언어를 확대해 기존 파이썬에서 자바까지 채용 범위를 확대키로 했다. 이를 통해 신규 사업 등을 자바 언어를 기반으로도 개발할 예정이다. 이번 채용은 신입 및 경력 자바 서버 개발자를 대상으로 진행되며, 자바 개발 경험과 요기요 서비스에 관심 있는 개발자라면 누구나 지원 가능하다. 모집 기간은 오는 17일까지로, 지원서 접수 후 온라인 코딩테스트가 진행된다. 이후 테스트 결과에 따라 면접 전형을 진행해 최종 합격자를 선발할 예정이다. 면접은 코로나 19를 고려해 비대면 방식으로 진행된다. 모집 과정 및 일정에 대한 자세한 내용은 딜리버리히어로 코리아 테크 홈페이지를 통해 확인 가능하다. 조현준 딜리버리히어로 코리아 최고기술책임자(CTO)는 “파이썬 위주의 개발 환경에서 벗어나 자바 개발자도 성장할 수 있는 개발 조직으로 나아가기 위해 이번 채용을 확대, 진행하게 됐다”면서 “이번 채용을 통해 합류할 우수 인재들은 앞으로 요기요가 전개해 나갈 다양한 행보들을 함께 고민하고 성장해 나갈 수 있어 매우 좋은 성장의 기회가 될 수 있을 것으로 기대한다”고 밝혔다. 한편, 요기요는 3년 내 최대 1000명 규모로 R&D 조직을 확대하고, 요기요 만의 차별화된 기술 경쟁력을 갖춰 나간다는 계획이다.    // 본문 내용   ']


In [23]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

1
1


In [24]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)